# Get all the URLs for each car dealer in Berlin

In [8]:
import asyncio
from playwright.async_api import async_playwright

async def visit_autoscout24_dealers():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        all_dealer_urls = []
        
        # Handle cookie consent on first page
        url = "https://www.autoscout24.de/haendler/#?city=Berlin&pageIndex=1&sortBy=distanceAscending"
        await page.goto(url)
        await page.wait_for_selector("h1", timeout=10000)
        
        # Accept cookies
        accept_button = await page.wait_for_selector('button[data-testid="as24-cmp-accept-all-button"]')
        await accept_button.click()
        
        # Iterate through all pages
        for page_index in range(1, 83):  # Pages 1 to 82
            url = f"https://www.autoscout24.de/haendler/#?city=Berlin&pageIndex={page_index}&sortBy=distanceAscending"
            await page.goto(url)
            
            # Wait for dealer list items to load
            await page.wait_for_selector('div[data-react-component="DealerListItem"]')
            
            # Extract dealer URLs from current page
            dealer_urls = await page.evaluate('''
                () => {
                    const dealers = document.querySelectorAll('div[data-react-component="DealerListItem"] .name a');
                    return Array.from(dealers).map(a => a.href);
                }
            ''')
            
            all_dealer_urls.extend(dealer_urls)
            print(f"Page {page_index}: Found {len(dealer_urls)} dealers")
            
            # Optional: Add a small delay between pages to avoid overwhelming the server
            await page.wait_for_timeout(1000)
        
        await browser.close()
        print(f"Total dealers found: {len(all_dealer_urls)}")
        return all_dealer_urls

# For Jupyter notebook
dealer_urls = await visit_autoscout24_dealers()

Page 1: Found 10 dealers
Page 2: Found 10 dealers
Page 3: Found 10 dealers
Page 4: Found 10 dealers
Page 5: Found 10 dealers
Page 6: Found 10 dealers
Page 7: Found 10 dealers
Page 8: Found 10 dealers
Page 9: Found 10 dealers
Page 10: Found 10 dealers
Page 11: Found 10 dealers
Page 12: Found 10 dealers
Page 13: Found 10 dealers
Page 14: Found 10 dealers
Page 15: Found 10 dealers
Page 16: Found 10 dealers
Page 17: Found 10 dealers
Page 18: Found 10 dealers
Page 19: Found 10 dealers
Page 20: Found 10 dealers
Page 21: Found 10 dealers
Page 22: Found 10 dealers
Page 23: Found 10 dealers
Page 24: Found 10 dealers
Page 25: Found 10 dealers
Page 26: Found 10 dealers
Page 27: Found 10 dealers
Page 28: Found 10 dealers
Page 29: Found 10 dealers
Page 30: Found 10 dealers
Page 31: Found 10 dealers
Page 32: Found 10 dealers
Page 33: Found 10 dealers
Page 34: Found 10 dealers
Page 35: Found 10 dealers
Page 36: Found 10 dealers
Page 37: Found 10 dealers
Page 38: Found 10 dealers
Page 39: Found 10 dea

In [9]:
print(dealer_urls)

['https://www.autoscout24.de/haendler/carpoint-gmbh-berlin', 'https://www.autoscout24.de/haendler/pkw-online-de-neuwagen', 'https://www.autoscout24.de/haendler/taheri-auto-kassel', 'https://www.autoscout24.de/haendler/motorrad-lukas', 'https://www.autoscout24.de/haendler/suv4you-gmbh', 'https://www.autoscout24.de/haendler/g-m-automobile-berlin', 'https://www.autoscout24.de/haendler/duerkop-gmbh-berlin-prenzlauer-berg-berlin', 'https://www.autoscout24.de/haendler/amore-automobile-moresco-e-k', 'https://www.autoscout24.de/haendler/duerkop-gmbh-berlin-10409', 'https://www.autoscout24.de/haendler/red-white-corporation-gmbh', 'https://www.autoscout24.de/haendler/carpoint-gmbh-berlin', 'https://www.autoscout24.de/haendler/pkw-online-de-neuwagen', 'https://www.autoscout24.de/haendler/taheri-auto-kassel', 'https://www.autoscout24.de/haendler/motorrad-lukas', 'https://www.autoscout24.de/haendler/suv4you-gmbh', 'https://www.autoscout24.de/haendler/g-m-automobile-berlin', 'https://www.autoscout24

In [11]:
# Write URLs to a file, one URL per line, adding "/ueber-uns" to each URL
with open('dealer_urls.txt', 'w') as f:
    for url in dealer_urls:
        # Remove trailing slash if present and add "/ueber-uns"
        modified_url = url.rstrip('/') + '/ueber-uns'
        f.write(f"{modified_url}\n")

print(f"Saved {len(dealer_urls)} URLs to dealer_urls.txt")

Saved 820 URLs to dealer_urls.txt


In [21]:
#next i need to get the number, person, opening hours, email, url
async def collect_dealer_info():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        dealer_info = []
        
        # Read URLs from the file
        with open('dealer_urls.txt', 'r') as f:
            urls = f.readlines()
        
        for url in urls:
            url = url.strip()
            try:
                await page.goto(url)
                
                # Get the dealer name
                name_element = await page.wait_for_selector('h1.sc-font-xl')
                dealer_name = await name_element.inner_text() if name_element else "N/A"
                
                # Get the phone number
                phone_element = await page.wait_for_selector('div.dp-top__phone a[data-testid="contact-person-link"]')
                phone_number = await phone_element.inner_text() if phone_element else "N/A"
                
                # Get the contact person name and email
                contact_info = await page.evaluate('''
                    () => {
                        const contactWrapper = document.querySelector('div[data-testid="contact-person-wrapper"]');
                        if (contactWrapper) {
                            const paragraphs = contactWrapper.getElementsByTagName('p');
                            const emailLink = contactWrapper.querySelector('a[data-testid="email-link"]');
                            return {
                                name: paragraphs[1]?.textContent || "N/A",
                                email: emailLink ? emailLink.getAttribute('href').replace('mailto:', '') : "N/A"
                            };
                        }
                        return { name: "N/A", email: "N/A" };
                    }
                ''')
                
                # Get opening hours
                opening_hours = await page.evaluate('''
                    () => {
                        const hours = {};
                        const container = document.querySelector('.dp-opening-hours-block__container');
                        if (container) {
                            const dts = container.querySelectorAll('dt.dp-opening-hours__label');
                            const dds = container.querySelectorAll('dd.dp-opening-hours__list');
                            
                            for (let i = 0; i < dts.length; i++) {
                                const day = dts[i].textContent.trim();
                                const timeElement = dds[i];
                                
                                if (timeElement.querySelector('.sc-font-silent')) {
                                    hours[day] = "Geschlossen";
                                } else {
                                    const times = Array.from(timeElement.querySelectorAll('span[data-testid="dp-opening-hours-opened-day"]'))
                                        .map(span => span.textContent.trim());
                                    hours[day] = times.join('–').replace('Uhr', '').trim();
                                }
                            }
                        }
                        return hours;
                    }
                ''')
                
                dealer_info.append({
                    'url': url,
                    'name': dealer_name,
                    'phone': phone_number,
                    'contact_person': contact_info['name'],
                    'email': contact_info['email'],
                    'opening_hours': opening_hours
                })
                
                print(f"Collected: {dealer_name} - {contact_info['name']} - {phone_number} - {contact_info['email']}")
                print(f"Opening hours: {opening_hours}")
                
                # Optional: Add a small delay between requests
                await page.wait_for_timeout(1000)
                
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
                continue
            break
        
        await browser.close()
        return dealer_info

# For Jupyter notebook
dealer_info = await collect_dealer_info()


Collected: Carpoint GmbH - Maik Jung - 030-57714957 - info@carpoint-nrw.de
Opening hours: {'Mo – Fr': '09:00––18:00', 'Sa': '10:00––14:00', 'So': 'Geschlossen'}
